In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException, ElementNotInteractableException, NoSuchElementException
import re
import pandas as pd
import time 
import jellyfish as jf

from typing import Callable

In [39]:
!pip install chromedriver-py

In [4]:
from chromedriver_py import binary_path # this will get you the path variable

# driver = webdriver.Chrome()

options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--window-size=1920x1080')
chrome_options.add_argument('--disable-gpu')
# chrome_options.add_argument('--single-process')
chrome_options.add_argument('--user-data-dir={}'.format(self._tmp_folder + '/user-data'))
chrome_options.add_argument('--hide-scrollbars')
chrome_options.add_argument('--enable-logging')
chrome_options.add_argument('--log-level=0')
chrome_options.add_argument('--v=99')
chrome_options.add_argument('--data-path={}'.format(self._tmp_folder + '/data-path'))
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--homedir={}'.format(self._tmp_folder))
chrome_options.add_argument('--disk-cache-dir={}'.format(self._tmp_folder + '/cache-dir'))
# options.add_argument('--headless')
# options.add_argument("window-size=1920x1080")
driver = webdriver.Chrome(chrome_options=options)

<ipython-input-4-d1689756ca71>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


In [ ]:
    def close_modal(self):
        close_button = self.driver.find_elements_by_id("naoRedirecionaInfoCredit")
        if len(close_button):
            try:
                close_button[0].click()
            except:
                pass

In [10]:
driver.find_elements_by_id("naoAbreAbaSCR")

[<selenium.webdriver.remote.webelement.WebElement (session="b7d5fd93f117e64dca9595c39a1d54ba", element="7e93a6a9-b97b-429a-95db-4185b6956f21")>]

In [11]:
def close_modal1():
    close_button = driver.find_elements_by_id("naoAbreAbaSCR")
    if len(close_button):
        try:
            close_button[0].click()
        except:
            pass
        
close_modal1()

In [35]:
class VaduCrawler():
    def __init__(self, cnpj: Callable[[], str]):
        self.USUARIO = "daniel@sthima.com.br"
        self.SENHA = "Score@123"
        self.TEMPO_ATT_PROTESTO = 3
        self.NUMBER_OF_TRY = 3
        self.VADU_SITE_HOST = "https://www.vadu.com.br/vadu.dll/Autenticacao/Entrar"


        if type(cnpj) == type([]):
            self.cnpj = [re.sub(r'[^\w\s]','',i) for i in cnpj]
        else:
            self.cnpj = [re.sub(r'[^\w\s]','',cnpj)]


    def capture_data(self):
        result = self.fill_info_vector(self.cnpj , 0, [])
        df_final = pd.DataFrame() 
        for i in result:
            df_final = df_final.append(pd.DataFrame([i]), ignore_index = True)
        
        if 0 in df_final.columns:
            self.df_final = df_final.drop(index = df_final[df_final[0].notnull()].index, columns = [0]).reset_index(drop = True)
        else:
            self.df_final = df_final

        return self.df_final


    def open_vadu_web_site(self):
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument("window-size=1920x1080")
        driver = webdriver.Chrome(chrome_options=options)
        driver.get(self.VADU_SITE_HOST)
        return driver

    def close_modal1(self):
        close_button = self.driver.find_elements_by_id("naoAbreAbaSCR")
        if len(close_button):
            try:
                close_button[0].click()
            except:
                pass
        
    def close_modal2(self):
        close_button = self.driver.find_elements_by_id("naoRedirecionaInfoCredit")
        if len(close_button):
            try:
                close_button[0].click()
            except:
                pass

    def make_login(self):
        user_input = self.driver.find_element_by_name('Codigo')
        key_input = self.driver.find_element_by_name('Senha')
        
        user_input.clear()
        user_input.send_keys(self.USUARIO)
        
        key_input.clear()
        key_input.send_keys(self.SENHA)
        
        self.driver.find_element_by_css_selector("button.btn").click()

    def login(self):      
        self.driver.get(self.VADU_SITE_HOST)
        if len(self.driver.find_elements_by_class_name('g-recaptcha-outer')):
            self.driver.close()
            assert 

            self.driver = self.open_vadu_web_site()
        
        self.make_login()


    def search_cnpj(self, cnpj):
        cnpj_input = self.driver.find_element_by_name('q')
        try:
            cnpj_input.clear()
        except:
            pass
        cnpj_input.send_keys(cnpj)
        cnpj_input.send_keys(Keys.ENTER)

    def necessary_update(self):
        last_update = self.driver.find_element_by_class_name('fonte-atualizacao.protestosConsultaDe').text.strip()
        
        def separet_number_and_letters(s):
            numbers = []
            text = []

            for i in s:
                try:
                    float(i)
                    numbers.append(i)
                except:
                    text.append(i)
                    
            try:
                number = float(''.join(numbers))
            except:
                number = 0

            return number, ''.join(text)
        
        time1, time2 = separet_number_and_letters(last_update)
        
        if 'mes' in time2 and time1 >= self.TEMPO_ATT_PROTESTO:
            return True
        elif 'ano' in time2:
            return True
        elif 'não consultado' in time2:
            return True
        else:
            return False


    def wait_load(self, delay = 60):
        try:
            start = time.time()
            while self.driver.find_element_by_class_name('totalProtestos').text == '' or (time.time() - start) > delay:
                self.driver.find_element_by_xpath('//a[@href="#tabProtestos"]').click()
                if self.driver.find_element_by_class_name('totalProtestos').text == '?':
                    break

            return True
        except TimeoutException:
            return False
        
    def wait_load_button(self, delay = 60):
        try:
            element = WebDriverWait(self.driver,delay).until(EC.element_to_be_clickable((By.ID, "btnAcaoAtualizarProtestoV2")))
            return True
        except TimeoutException:
            return False

    def DEFAULT_OBJ(self, cnpj,faturamento = None,\
                               funcionarios = None,\
                               total_protesto = None,\
                               valor_protesto = None):

            return {'cnpj':cnpj,
                'Faturamento':faturamento,
                'Funcionarios':funcionarios,
                'TotalProtesto':total_protesto,
                'ValorProtesto':valor_protesto}

    def creat_possibles_names_df(self):
        cnpjs=[]
        nomes=[]

        tables=self.driver.find_elements_by_xpath('//*[@id="tableCadastro"]/tbody/tr')
        for table in tables:
            aux = table.find_elements_by_xpath("./td")
            cnpjs.append(aux[0])
            nomes.append(aux[1].text)
            
        return pd.DataFrame({'cnpjs':cnpjs, 'nomes':nomes})

    def select_best_name(self, cnpj):
        possiveis_cnpjs = self.creat_possibles_names_df()
        possiveis_cnpjs['nomes'] = possiveis_cnpjs['nomes'].apply(lambda x: re.sub(' +', ' ',re.sub(r'[^\w\s]','',x)).strip())
        possiveis_cnpjs['dist_cnpj'] = possiveis_cnpjs['nomes'].apply(lambda x: jf.levenshtein_distance(x, cnpj)) 
        possiveis_cnpjs = possiveis_cnpjs.sort_values('dist_cnpj').reset_index(drop = True)
        possiveis_cnpjs['cnpjs'].iloc[0].click()
        
    def capture_VADU_info(self, cnpj):
        self.driver.get(self.VADU_SITE_HOST)
        self.login()
        self.close_modal()

        self.search_cnpj(cnpj)
        

        if len(self.driver.find_elements_by_xpath('//*[@id="tableCadastro"]/tbody/tr')):
            self.select_best_name(cnpj)
        
        result_search = self.driver.find_elements_by_id('ResultadoBusca')
        if len(result_search):
            if result_search.text == 'Pesquisa sem resultados':
                print('DEU ERRO 1')
                return self.DEFAULT_OBJ(cnpj)
            
        cnpj = self.driver.find_element_by_id('cnpjBlocoDinamico').text
        faturamento = self.driver.find_element_by_id('valorFaturamentoEstimado').text
        funcionarios = self.driver.find_element_by_id('valorQuantidadeFuncionarios').text
        
        protest_button = WebDriverWait(self.driver,5).until(EC.presence_of_element_located((By.XPATH, '//a[@href="#tabProtestos"]')))
        protest_button.click()

        if not(self.wait_load()):
            print('DEMOROU DMAIS')
            return self.DEFAULT_OBJ(cnpj)
        
        if self.necessary_update():
            self.driver.find_element_by_id('btnAcaoAtualizarProtestoV2').click()
            if not(self.wait_load_button()):
                print('DEU ERRO 2')
                return self.DEFAULT_OBJ(cnpj)
        
        total_protesto = self.driver.find_element_by_class_name('totalProtestos').text
        valor_protesto = self.driver.find_element_by_class_name('valor-total-protestos').text.replace('R$','').strip()
        self.driver.close()
        
        return self.DEFAULT_OBJ(cnpj,faturamento,funcionarios,total_protesto,valor_protesto)


    def fill_info_vector(self, cnpj_vector, i, vadu_infos, n_try = 0 ):
        if i >= len(cnpj_vector):
            return vadu_infos
        else:
            self.driver = self.open_vadu_web_site()
            try:
                vadu_infos.append(self.capture_VADU_info(cnpj_vector[i]))
                return self.fill_info_vector( cnpj_vector, i+1, vadu_infos)
            except Exception as e:
                print(str(e))
                try:
                    self.driver.close()
                except:
                    pass
                n_try += 1
                if n_try >= self.NUMBER_OF_TRY:
                    vadu_infos.append(cnpj_vector[i])
                    return self.fill_info_vector( cnpj_vector, i+1, vadu_infos)
                else:
                    return self.fill_info_vector( cnpj_vector, i, vadu_infos, n_try)


        


In [37]:
aux = VaduCrawler('89949796000137').capture_data()
aux

<ipython-input-35-7bf301977428>:34: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


,cnpj,Faturamento,Funcionarios,TotalProtesto,ValorProtesto
0,89.949.796/0001-37,de 1.500.000 até 4.800.000,de 10 até 50,20,"514.443,80"


In [27]:

# driver = webdriver.Chrome(chrome_options=options)
driver.get("https://www.vadu.com.br/vadu.dll/Autenticacao/Entrar/43324124?saiu=sim")

In [28]:
USUARIO = "daniel@sthima.com.br"
SENHA = "Score@123"
TEMPO_ATT_PROTESTO = 3


In [29]:
def make_login(driver):
    user_input = driver.find_element_by_name('Codigo')
    key_input = driver.find_element_by_name('Senha')
    
    user_input.clear()
    user_input.send_keys(USUARIO)
    
    key_input.clear()
    key_input.send_keys(SENHA)
    
    driver.find_element_by_css_selector("button.btn").click()

def login(driver):
    try:
        driver.get("https://www.vadu.com.br/vadu.dll/Autenticacao/Entrar/43324124?saiu=sim")
        driver.find_element_by_xpath('//*[@id="barra-icones"]/li[6]/a').click()
    except:
        pass
        
        
    driver.get("https://www.vadu.com.br/vadu.dll/Autenticacao/Entrar/43324124?saiu=sim")
    if len(driver.find_elements_by_class_name('g-recaptcha-outer')):
        driver.close()
        
        aux_driver = webdriver.Firefox()
        aux_driver.get("https://www.vadu.com.br/vadu.dll/Autenticacao/Entrar/43324124?saiu=sim")
        make_login(aux_driver)
        close_modal(aux_driver)
        aux_driver.close()
        
        driver = webdriver.Chrome()
        driver.get("https://www.vadu.com.br/vadu.dll/Autenticacao/Entrar/43324124?saiu=sim")
    
    make_login(driver)
    
    return driver

In [30]:
login(driver)

<selenium.webdriver.chrome.webdriver.WebDriver (session="f9ca38614b79d9f09c962531a7d0ce7d")>

In [31]:
def close_modal(driver):
    close_button = driver.find_elements_by_id("naoRedirecionaInfoCredit")
    if len(close_button):
        try:
            close_button[0].click()
        except:
            pass

In [32]:
close_modal(driver)

In [33]:
capture_VADU_info(driver, '89949796000137')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="tabProtestos"]/div/div[4]/div/div[5]/h5"}
  (Session info: headless chrome=91.0.4472.114)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="tabProtestos"]/div/div[4]/div/div[5]/h5"}
  (Session info: headless chrome=91.0.4472.114)


In [21]:
cnpj_input = driver.find_element_by_name('q')

In [24]:
cnpj_input.send_keys('89949796000137')

ElementNotInteractableException: Message: element not interactable
  (Session info: headless chrome=91.0.4472.114)


In [127]:
    cnpj = driver.find_element_by_id('cnpjBlocoDinamico').text
    faturamento = driver.find_element_by_id('valorFaturamentoEstimado').text
    funcionarios = driver.find_element_by_id('valorQuantidadeFuncionarios').text

In [15]:


def search_cnpj(driver, cnpj):
    cnpj_input = driver.find_element_by_name('q')
    try:
        cnpj_input.clear()
    except:
        pass
    cnpj_input.send_keys(cnpj)
    cnpj_input.send_keys(Keys.ENTER)
    


In [16]:
def necessary_update(driver):
    last_update = driver.find_element_by_xpath('//*[@id="tabProtestos"]/div/div[4]/div/div[5]/h5').text.strip()
    
    def separet_number_and_letters(s):
        numbers = []
        text = []

        for i in s:
            try:
                float(i)
                numbers.append(i)
            except:
                text.append(i)
                
        try:
            number = float(''.join(numbers))
        except:
            number = 0

        return number, ''.join(text)
    
    time1, time2 = separet_number_and_letters(last_update)
    
    if 'mes' in time2 and time1 >= TEMPO_ATT_PROTESTO:
        return True
    elif 'ano' in time2:
        return True
    elif 'não consultado' in time2:
        return True
    else:
        return False



In [135]:
driver.find_element_by_id('btnAcaoAtualizarProtestoV2')

<selenium.webdriver.remote.webelement.WebElement (session="cc5fba688efe70c51517bd9c52138ecf", element="083dc0fd-5d2d-40ce-a211-5a876231d90c")>

In [19]:
def wait_load(driver, delay = 60):
    try:
        start = time.time()
        while driver.find_element_by_class_name('valor-total-protestos').text == '' or (time.time() - start) > delay:
            driver.find_element_by_xpath('//a[@href="#tabProtestos"]').click()
            if driver.find_element_by_class_name('totalProtestos').text == '?':
                break

        return True
    except TimeoutException:
        return False
    
def wait_load_button(driver, delay = 60):
    try:
        element = WebDriverWait(driver,delay).until(EC.element_to_be_clickable((By.ID, "btnAcaoAtualizarProtesto")))
        return True
    except TimeoutException:
        return False

def DEFAULT_OBJ(cnpj):
        return {'cnpj':cnpj,
            'faturamento':np.nan,
            'funcionarios':np.nan,
            'total_protesto':np.nan,
            'valor_protesto':np.nan} 
    
    
def creat_possibles_names_df():
    cnpjs=[]
    nomes=[]

    tables=driver.find_elements_by_xpath('//*[@id="tableCadastro"]/tbody/tr')
    for table in tables:
        aux = table.find_elements_by_xpath("./td")
        cnpjs.append(aux[0])
        nomes.append(aux[1].text)
        
    return pd.DataFrame({'cnpjs':cnpjs, 'nomes':nomes})

def select_best_name(cnpj):
    possiveis_cnpjs = creat_possibles_names_df()
    possiveis_cnpjs['nomes'] = possiveis_cnpjs['nomes'].apply(lambda x: re.sub(' +', ' ',re.sub(r'[^\w\s]','',x)).strip())
    possiveis_cnpjs['dist_cnpj'] = possiveis_cnpjs['nomes'].apply(lambda x: jf.levenshtein_distance(x, cnpj)) 
    possiveis_cnpjs = possiveis_cnpjs.sort_values('dist_cnpj').reset_index(drop = True)
    possiveis_cnpjs['cnpjs'].iloc[0].click()
    
    
    
def capture_VADU_info(driver, cnpj):
    driver.get("https://www.vadu.com.br/vadu.dll/Autenticacao/Entrar/43324124?saiu=sim")
    driver = login(driver)
    close_modal(driver)

    search_cnpj(driver, cnpj)
    result_search = driver.find_elements_by_id('ResultadoBusca')
    
    if len(driver.find_elements_by_xpath('//*[@id="tableCadastro"]/tbody/tr')):
        select_best_name('cnpj')
        
        
    if len(result_search):
        if result_search.text == 'Pesquisa sem resultados':
            return DEFAULT_OBJ(cnpj)
        
    cnpj = driver.find_element_by_id('cnpjBlocoDinamico').text
    faturamento = driver.find_element_by_id('valorFaturamentoEstimado').text
    funcionarios = driver.find_element_by_id('valorQuantidadeFuncionarios').text
    
    protest_button = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH, '//a[@href="#tabProtestos"]')))
    protest_button.click()
    if not(wait_load(driver)):
            return DEFAULT_OBJ(cnpj)
    
    if necessary_update(driver):
        driver.find_element_by_id('btnAcaoAtualizarProtesto').click()
        if not(wait_load_button(driver)):
            return DEFAULT_OBJ(cnpj)
    
    total_protesto = driver.find_element_by_class_name('totalProtestos').text
    valor_protesto = driver.find_element_by_class_name('valor-total-protestos').text.replace('R$','').strip()
    driver.close()
    
    return {'cnpj':cnpj,
            'faturamento':faturamento,
            'funcionarios':funcionarios,
            'total_protesto':total_protesto,
            'valor_protesto':valor_protesto}

In [41]:
import pymongo
import pandas as pd
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["ATFCredit"]
mycol = mydb['Txt_features']
cursor = mycol.find({'result': {'$ne': -1}})
df = pd.DataFrame(list(cursor))

[]

In [121]:

def fill_info_vector(driver, cnpj_vector, i, vadu_infos, n_try = 0 ):
    if i >= len(cnpj_vector):
        return vadu_infos, i
    else:
        driver = webdriver.Chrome(chrome_options=options)
        try:
            vadu_infos.append(capture_VADU_info(driver,cnpj_vector[i]))
            return fill_info_vector(driver, cnpj_vector, i+1, vadu_infos)
        except Exception as e:
            try:
                driver.close()
            except:
                pass
            n_try += 1
            if n_try >= 3:
                vadu_infos.append(cnpj_vector[i])
                return fill_info_vector(driver, cnpj_vector, i+1, vadu_infos)
            else:
                return fill_info_vector(driver, cnpj_vector, i, vadu_infos, n_try)
    

In [123]:
vadu_infos = []
result, ind = fill_info_vector(driver, df['cnpj'], 70, [])

<ipython-input-121-abf577c39365>:19: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


In [133]:
df_final = pd.DataFrame() 
for i in result:
    df_final = df_final.append(pd.DataFrame([i]), ignore_index = True)




In [145]:
df_final = df_final.drop(index = df_final[df_final[0].notnull()].index, columns = [0]).reset_index(drop = True)

In [147]:

df_final.to_csv('FIM.csv')

In [15]:
result[0]

[{'cnpj': '01.314.776/0001-49',
  'faturamento': 'de 1.500.000 até 4.800.000',
  'funcionarios': 'de 10 até 50',
  'total_protesto': '36',
  'valor_protesto': '9.196.456,58'},
 {'cnpj': '00.061.315/0001-49',
  'faturamento': 'de 100.000.000 até 300.000.000',
  'funcionarios': 'de 50 até 100',
  'total_protesto': '2',
  'valor_protesto': '64.680,62'},
 {'cnpj': '00.863.529/0001-39',
  'faturamento': 'de 10.000.000 até 30.000.000',
  'funcionarios': 'de 50 até 100',
  'total_protesto': '0',
  'valor_protesto': '-'},
 {'cnpj': '83.109.579/0001-52',
  'faturamento': 'de 10.000.000 até 30.000.000',
  'funcionarios': 'de 100 até 500',
  'total_protesto': '21',
  'valor_protesto': '28.991.636,43'},
 {'cnpj': '88.371.075/0001-20',
  'faturamento': 'de 1.500.000 até 4.800.000',
  'funcionarios': 'de 10 até 50',
  'total_protesto': '0',
  'valor_protesto': '-'},
 {'cnpj': '86.837.366/0001-35',
  'faturamento': 'de 30.000.000 até 100.000.000',
  'funcionarios': 'de 100 até 500',
  'total_protesto

In [5]:
df1 = pd.read_csv('FIM.csv',index_col = 0)
df2 = pd.read_csv('0-70_cnpjs.csv',index_col = 0)

In [9]:
df = pd.concat([df1,df2])

In [11]:
df.to_csv('Base_Crawler.csv')